In [4]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, "../")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.ConfigSer import ConfigSer

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [6]:
cmap='jet'
configFilenameWr15 = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueSupport.ini"
configFilenameWr10 = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueSupport.ini"
conf15 = ConfigSer(configFilenameWr15)
conf10 = ConfigSer(configFilenameWr10)
conf15.printConfig()
conf10.printConfig()

C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueSupport.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.7
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr15
gateWidth 500.0
gateCenter 6650.0
C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueSupport.ini
nb_elev 41
nb_freq 2801
nb_ssb 4
el0 0.65
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr10
gateWidth 500.0
gateCenter 6650.0


In [13]:
dirWR15 = f"{conf15.out_path}/data"
dirWR10 = f"{conf10.out_path}/data"
prefix = f"{conf10.base_path}/processing/wr15_wr10/"

In [9]:
iWR15 = 20
iWR10 = 21
freq_WR15 = np.load( f"{dirWR15}/freq_WR15.npy" )
freq_WR10 = np.load( f"{dirWR10}/freq_WR10.npy" )
elevation_WR15 = np.load( f"{dirWR15}/elevation_WR15.npy" )
elevation_WR10 = np.load( f"{dirWR10}/elevation_WR10.npy" )

# Compare WR15 / WR10

In [15]:
fig, ax = plt.subplots(1,1)
title = prefix

getMax = 0

dictOfMeasurements = {"C": (("20200701", "001_C"), (20200709", "037_C"))}

for myAbs in dictOfMeasurements["C"]:
    date, absorbant = myAbs
    if int(date) < 20200709:
        # this is a WR15 measurement
        reflectivite = np.load( f"{dirWR15}/reflectivite/{absorbant}_{date}.npy" )
        toPlot = reflectivite[iWR15, :]
        if getMax:
            toPlot = np.amax(np.abs(reflectivite), axis=0)
        ax.plot( freq_WR15, ser.dB( toPlot ), 
                label = f"WR15, {absorbant}, {date}, elevation = {elevation_WR15[iWR15]:.2f}" )
    else:
        # this is a WR10 measurement
        reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" )  
        toPlot = reflectivite[iWR10, :]
        if getMax:
            toPlot = np.amax(np.abs(reflectivite), axis=0)
        ax.plot( freq_WR10, ser.dB( toPlot ), 
                label = f"WR10, {absorbant}, {date}, elevation = {elevation_WR10[iWR10]:.2f}" )
    if title == prefix:
        if getMax:
            title = prefix + f"max/{absorbant}_{date}"
        else:
            title = prefix + f"{absorbant}_{date}"
    else:
        title = title + f"_{absorbant}_{date}"

ax.legend()
ax.grid()
ax.set_xlabel("[GHz]")
ax.set_ylabel("[dB]")
ax.set_title(f"Réflectivité")
if getMax:
    title += "_max"
fig.savefig(f"{title}.png")

# Compare all

In [ ]:
for root, dirs, files in os.walk(prefix):
    for f in files:
        if f.endswith(".png"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same

In [ ]:
fig, ax = plt.subplots(1,1)

getMax = 0

for item in wr15_wr10_dates.dictOfMeasurements.items():
    ref, tests = item
    title = prefix
    fig, ax = plt.subplots(1,1)
    print(f"{ref}, {len(tests)} measurements")
    for test in tests:
        serie, date, absorbant = test
        if int(date) < 190910:
            # this is a WR15 measurement
            reflectivite = np.load( f"{dirWR15}/reflectivite_abs{absorbant}.npy" )
            toPlot = reflectivite[iWR15, :]
            if getMax:
                toPlot = np.amax(np.abs(reflectivite), axis=0)
            ax.plot( freq_WR15, ser.dB( toPlot ), 
                    label = f"WR15, {absorbant}, {date}, elevation = {elevation_WR15[iWR15]:.2f}" )
        else:
            # this is a WR10 measurement
            reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" )    
            toPlot = reflectivite[iWR10, :]
            if getMax:
                toPlot = np.amax(np.abs(reflectivite), axis=0)
            ax.plot( freq_WR10, ser.dB( toPlot ), 
                    label = f"WR10, {absorbant}, {date}, elevation = {elevation_WR10[iWR10]:.2f}, serie {serie}" )
        if getMax:
            title = prefix + f"max/{absorbant}_{date}"
        else:
            title = prefix + f"elev_i/{absorbant}_{date}"
    ax.legend()
    ax.grid()
    ax.set_xlabel("[GHz]")
    ax.set_ylabel("[dB]")
    ax.set_title(f"Réflectivité")
    if getMax:
        title += "_max"
    fig.savefig(f"{title}.png")
    plt.close()

# Plot cmap

## One

In [ ]:
fig, ax = plt.subplots(1,1)
vmin = -90
vmax = -40
cmap='jet'
# wr15
centerFreqWR15 = 62.5
bandWidthWR15 = 12.4
idxWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) < bandWidthWR15 )
idxAltWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) >= bandWidthWR15 )
# wr10
centerFreqWR10 = 92.5
bandWidthWR10 = 17.4
idxWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) < bandWidthWR10 )
idxAltWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) >= bandWidthWR10 )

refAbs = "AHP12"
meas0 = wr15_wr10_dates.dictOfMeasurements[refAbs][0]
meas1 = wr15_wr10_dates.dictOfMeasurements[refAbs][-1]
for meas in meas0, meas1:
    serie, date, absorbant = meas
    if int(date) < 190910:
        # this is a WR15 measurement
        reflectivite = np.load( f"{dirWR15}/reflectivite_abs{absorbant}.npy" )
        # drop lower and higher frequencies values
        toPlot = np.zeros(reflectivite.shape)
        toPlot[:,idxWR15]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR15] ) )
        toPlot[:,idxAltWR15] = np.nan
        im = ax.pcolormesh( elevation_WR15, np.flip( freq_WR15 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    else:
        # this is a WR10 measurement
        reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" ) 
        # drop lower and higher frequencies values
        toPlot = np.zeros(reflectivite.shape)
        toPlot[:,idxWR10]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR10] ) )
        toPlot[:,idxAltWR10] = np.nan
        im = ax.pcolormesh( elevation_WR10, np.flip( freq_WR10 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    
title = f"Réflectivité {refAbs}: {meas0[1]} / {meas1[1]}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )
fig.savefig(f"{prefix}/pcolormesh/reflectivite_{refAbs}.png")

## All

In [ ]:
vmin = -90
vmax = -40
cmap='jet'
# wr15
centerFreqWR15 = 62.5
bandWidthWR15 = 12.4
idxWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) < bandWidthWR15 )
idxAltWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) >= bandWidthWR15 )
# wr10
centerFreqWR10 = 92.5
bandWidthWR10 = 17.4
idxWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) < bandWidthWR10 )
idxAltWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) >= bandWidthWR10 )

for refAbs in wr15_wr10_dates.dictOfMeasurements.keys():
    fig, ax = plt.subplots(1,1)
    meas0 = wr15_wr10_dates.dictOfMeasurements[refAbs][0]
    meas1 = wr15_wr10_dates.dictOfMeasurements[refAbs][-1]
    for meas in meas0, meas1:
        serie, date, absorbant = meas
        if int(date) < 190910:
            # this is a WR15 measurement
            reflectivite = np.load( f"{dirWR15}/reflectivite_abs{absorbant}.npy" )
            # drop lower and higher frequencies values
            toPlot = np.zeros(reflectivite.shape)
            toPlot[:,idxWR15]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR15] ) )
            toPlot[:,idxAltWR15] = np.nan
            im = ax.pcolormesh( elevation_WR15, np.flip( freq_WR15 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
        else:
            # this is a WR10 measurement
            reflectivite = np.load( f"{dirWR10}/reflectivite_{absorbant}_{date}.npy" ) 
            # drop lower and higher frequencies values
            toPlot = np.zeros(reflectivite.shape)
            toPlot[:,idxWR10]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR10] ) )
            toPlot[:,idxAltWR10] = np.nan
            im = ax.pcolormesh( elevation_WR10, np.flip( freq_WR10 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    
    title = f"Réflectivité {refAbs}: {meas0[1]} / {meas1[1]}"
    ax.set_title(title)
    ax.set_xlabel( "Elévation [°]" )
    ax.set_ylabel( "Fréquence [GHz]" )
    dia.addColorBar(  im, ax, aspect='auto' )
    fig.savefig(f"{prefix}/pcolormesh/reflectivite_{refAbs}.png")
    plt.close()

# Remove all

In [ ]:
for root, dirs, files in os.walk(f"{prefix}"):
    for f in files:
        if f.endswith(".png") or f.endswith(".npy"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same